In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader, Sampler, RandomSampler, BatchSampler
from torch.utils.data.distributed import DistributedSampler
from torch.utils.tensorboard import SummaryWriter

import torch.backends.cudnn as cudnn

import torchaudio
import torchaudio.transforms as T

import os
import glob
import random
import numpy as np

from collections import OrderedDict

import json
from tqdm import tqdm


In [205]:
# path = "/Users/ujinne/python/Hackathon/speaker_recognition/data/train"
# path = "/Users/ujinne/python/Hackathon/speaker_recognition/data/sample/feature/train/"
path = "/Users/ujinne/python/Hackathon/speaker_recognition/data/sample/train/"

In [180]:
MelSpectrogram = T.MelSpectrogram(sample_rate=16000,
                                  n_fft=512,
                                  win_length=400,
                                  hop_length = 160,
                                  n_mels = 40)
    
AmplitudeToDB = T.AmplitudeToDB()

def wav2melspectrogram(filepath: str):
    samples, sample_rate = torchaudio.load(filepath)
    assert sample_rate == 16000, "Sampling Rate != 16000"

    mel_specgram = MelSpectrogram(samples)
    mel_specgram = AmplitudeToDB(mel_specgram)
    mel_specgram = mel_specgram.squeeze(0).transpose(0, 1).contiguous()

    return mel_specgram

In [206]:
# files = glob.glob(path + '/*')
# files = sorted([f for f in files if f != ".DS_Store"])

files = sorted([f.split('/')[-1] for f in glob.glob(path + '/*') if os.path.isdir(f)])
# len(f_list)
# files = files[:1000]

In [207]:
spk_id_list = []
for f in tqdm(files):
    spk_id = f.split('/')[-1]
    spk_id_list.append(spk_id)

100%|██████████| 3/3 [00:00<00:00, 42653.94it/s]


In [208]:
spk_id_list

['0CAE3COBqO', '6cmw8oKHST', '7K4iJllpGl']

In [204]:
batch = []
for i in range(len(spk_id_list)):
    spk_id = spk_id_list[i]
    utts = glob.glob(f'{path}/{spk_id}/*.pt')

    mel_spec_tensor_list = []
    # if len(utts) >= 10:
    #     utt_list = random.sample(utts, 10)
    # else:
    #     utt_list = random.choices(utts, k=10)

    # for utt in utt_list:
    for utt in utts:
        # print(utt)
        # mel_spectrogram_tensor = wav2melspectrogram(utt)
        mel_spec_tensor = torch.load(utt)
        print(mel_spec_tensor.shape)
        print(mel_spec_tensor)
        mel_spec_tensor_list.append(mel_spec_tensor)
    batch.append(mel_spec_tensor_list)

torch.Size([547, 40])
tensor([[-100., -100., -100.,  ..., -100., -100., -100.],
        [-100., -100., -100.,  ..., -100., -100., -100.],
        [-100., -100., -100.,  ..., -100., -100., -100.],
        ...,
        [-100., -100., -100.,  ..., -100., -100., -100.],
        [-100., -100., -100.,  ..., -100., -100., -100.],
        [-100., -100., -100.,  ..., -100., -100., -100.]])
torch.Size([717, 40])
tensor([[-100., -100., -100.,  ..., -100., -100., -100.],
        [-100., -100., -100.,  ..., -100., -100., -100.],
        [-100., -100., -100.,  ..., -100., -100., -100.],
        ...,
        [-100., -100., -100.,  ..., -100., -100., -100.],
        [-100., -100., -100.,  ..., -100., -100., -100.],
        [-100., -100., -100.,  ..., -100., -100., -100.]])
torch.Size([564, 40])
tensor([[-100., -100., -100.,  ..., -100., -100., -100.],
        [-100., -100., -100.,  ..., -100., -100., -100.],
        [-100., -100., -100.,  ..., -100., -100., -100.],
        ...,
        [-100., -100., 

In [209]:
batch = []
for i in range(len(spk_id_list)):
    spk_id = spk_id_list[i]
    utts = glob.glob(f'{path}/{spk_id}/*.wav')

    mel_spec_tensor_list = []
    # if len(utts) >= 10:
    #     utt_list = random.sample(utts, 10)
    # else:
    #     utt_list = random.choices(utts, k=10)

    # for utt in utt_list:
    for utt in utts:
        # print(utt)
        mel_spectrogram_tensor = wav2melspectrogram(utt)
        print(mel_spec_tensor.shape)
        print(mel_spec_tensor)
        mel_spec_tensor_list.append(mel_spec_tensor)
    batch.append(mel_spec_tensor_list)

torch.Size([666, 40])
tensor([[-100., -100., -100.,  ..., -100., -100., -100.],
        [-100., -100., -100.,  ..., -100., -100., -100.],
        [-100., -100., -100.,  ..., -100., -100., -100.],
        ...,
        [-100., -100., -100.,  ..., -100., -100., -100.],
        [-100., -100., -100.,  ..., -100., -100., -100.],
        [-100., -100., -100.,  ..., -100., -100., -100.]])
torch.Size([666, 40])
tensor([[-100., -100., -100.,  ..., -100., -100., -100.],
        [-100., -100., -100.,  ..., -100., -100., -100.],
        [-100., -100., -100.,  ..., -100., -100., -100.],
        ...,
        [-100., -100., -100.,  ..., -100., -100., -100.],
        [-100., -100., -100.,  ..., -100., -100., -100.],
        [-100., -100., -100.,  ..., -100., -100., -100.]])
torch.Size([666, 40])
tensor([[-100., -100., -100.,  ..., -100., -100., -100.],
        [-100., -100., -100.,  ..., -100., -100., -100.],
        [-100., -100., -100.,  ..., -100., -100., -100.],
        ...,
        [-100., -100., 

In [188]:
utt.split('/')[-1].replace(".wav", ".pt")

'zuqaTALtAS_1.pt'

In [52]:
sampling_frame_length = random.randint(140, 180)
melspec_list = []

for mel_spectrogram_tensor_list in batch:
    for utt_tensor in mel_spectrogram_tensor_list:
        len_time = utt_tensor.size(0)
        print(f'len_time: {len_time}')
        len_time = len_time - sampling_frame_length
        print(f'len_time_2: {len_time}')

        start_time = random.randint(0, len_time)
        end_time = start_time + sampling_frame_length
        
        melspec_list.append(utt_tensor[start_time:end_time, :])

mel_spec_tensors = torch.stack(melspec_list) # [batch_size, time_step, feature_dim] batch_size = N * M의 형태

len_time: 547
len_time_2: 398
len_time: 717
len_time_2: 568
len_time: 564
len_time_2: 415
len_time: 444
len_time_2: 295
len_time: 666
len_time_2: 517
len_time: 461
len_time_2: 312
len_time: 768
len_time_2: 619
len_time: 436
len_time_2: 287
len_time: 291
len_time_2: 142
len_time: 265
len_time_2: 116


In [62]:
mel_spec_tensors.shape

torch.Size([10, 149, 40])

In [78]:
class LSTM(nn.Module):
    def __init__(self,input_size=40, hidden_size=768, num_layers=3, batch_size=1):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.layers = nn.Linear(self.hidden_size, 256)
        self.batch_size = batch_size
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first = True)
 
    def forward(self, x):
#         print(x.shape)
        # x = x.squeeze(0)
        self.lstm.flatten_parameters()
        y,_ = self.lstm(x)
        y = self.layers(y[:,-1,:]) # (BS, T, emb_dim)
        
        y = y / torch.norm(y, p=2, dim=1, keepdim=True) # (BS, emb_dim)
        # y = y.sum(1) / y.size(1) # (emb_dim), average pooling over time frames
#         y = torch.mean(y, dim=1)
        
        return y
    
def get_LSTM_model(input_size, hidden_size, num_layers, batch_size):
    LSTM_model = LSTM(input_size, hidden_size, num_layers, batch_size)
    
    return LSTM_model

In [79]:
input_size = 40
hidden_size = 768
num_layers = 3
batch_size = 10

LSTM_model = get_LSTM_model(input_size, hidden_size, num_layers, batch_size)

In [73]:
lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
x = mel_spec_tensors[0].unsqueeze(0)
y, _ = lstm(x)

In [75]:
y.shape

torch.Size([1, 149, 768])

In [90]:
# x = x.unsqueeze(0)
x = mel_spec_tensors
print(x.shape)
dvecs = LSTM_model(x) # [batch_size, emb_dim]
print(dvecs.shape)

torch.Size([10, 149, 40])
torch.Size([10, 256])


batch_size는 N(화자의 수) * M(발화 개수) 였으므로 5*2 = 10이됨

그걸 view를 통해 [N, M, -1] shape으로 바꿔주면서 [5, 2, 256]이 되는 것 

In [96]:
dvecs = dvecs.view(5, 2, -1) # [N, M, emb_dim] 으로 바꿔야함
dvecs.shape

torch.Size([5, 2, 256])

In [97]:
torch.mean(dvecs, 1).shape

torch.Size([5, 256])

In [120]:
stepsize = 64
label = torch.from_numpy(np.asarray(range(0,stepsize)))
label

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63])

In [121]:
gsize = 10
target = torch.repeat_interleave(label,repeats=gsize,dim=0)
target

tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,
         5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,
         7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12,
        12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14,
        14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19,
        19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 21, 22, 22, 22, 22, 

In [128]:
type(target)

torch.Tensor

In [125]:
cos_sim_matrix = torch.rand(64, 10, 64)

In [129]:
type(cos_sim_matrix.view(-1, 64))

torch.Tensor

In [238]:
cos_sim_matrix.view(-1, 64).shape

torch.Size([640, 64])

In [248]:
def embed_loss_softmax(dvecs, cos_sim_matrix):
    '''
    Calculates the loss on each embedding $L(e_{ji})$ by taking softmax
    '''
    N, M, _ = dvecs.shape
    L = []
    for j in range(N):
        L_row = []
        for i in range(M):
            L_row.append(-F.log_softmax(cos_sim_matrix[j,i], 0)[j])
        L_row = torch.stack(L_row)
        L.append(L_row)
    return torch.stack(L)

In [246]:
dvecs = torch.rand(64, 10, 256)
dvecs.shape

torch.Size([64, 10, 256])

In [254]:
L = embed_loss_softmax(dvecs, cos_sim_matrix)
print(embed_loss_softmax(dvecs, cos_sim_matrix).shape)

torch.Size([64, 10])


In [256]:
L[0].sum()

tensor(42.4982)

In [134]:
criterion = torch.nn.CrossEntropyLoss()
loss = criterion(cos_sim_matrix.view(-1, stepsize), target)

In [244]:
loss

tensor(4.1911)

In [136]:
def  accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t() #transpose
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))

        return res

In [139]:
prec = accuracy(cos_sim_matrix.view(-1,stepsize).detach(), torch.repeat_interleave(label,repeats=gsize,dim=0).detach(), topk=(1,))[0]

In [140]:
prec

tensor([2.0312])

In [141]:
a = torch.rand(2,2,2)
a

tensor([[[0.7531, 0.1310],
         [0.1752, 0.2394]],

        [[0.7106, 0.2597],
         [0.2194, 0.1900]]])

In [142]:
a.view(-1, 2)

tensor([[0.7531, 0.1310],
        [0.1752, 0.2394],
        [0.7106, 0.2597],
        [0.2194, 0.1900]])

In [158]:
def split_train_val(train_dir, split_path, val_ratio=0.2):
    speakers = [x for x in train_dir.iterdir() if x.is_dir()]
    print("Found ", len(speakers), "speakers")
    wav_list = []
    split_list = []
    for id_ in tqdm(speakers):
        utterences = ['/'.join(x.parts[-2:]) for x in id_.glob("*.wav")]
        split_list.extend((np.random.rand(len(utterences)) > val_ratio).astype(int))
        wav_list.extend(utterences)
    # save split.txt
    with open(split_path, "w") as split:
        for i, n in enumerate(split_list):
            split.write("%d %s\n" % (n, wav_list[i]))

    print(len(split_list)," files split into train/val")

In [189]:
dir_path = "/Users/ujinne/python/Hackathon/speaker_recognition/data/train/"
split_path = '/Users/ujinne/python/Hackathon/speaker_recognition/data/split.txt'
# split_train_val(dir_path, split_path, val_ratio = 0.2)

In [191]:
save_dir = os.path.join(dir_path, "IDMXKDW")
utt = "eiwfjfklp.pt"

os.path.join(save_dir,utt)

'/Users/ujinne/python/Hackathon/speaker_recognition/data/train/IDMXKDW/eiwfjfklp.pt'

In [235]:
def split_train_val(train_dir, train_path, val_path, val_ratio=0.2):
    speakers = [x for x in train_dir.iterdir() if x.is_dir()]
    # speakers = sorted([f for f in glob.glob(train_dir + '/*') if os.path.isdir(f)])
    print("Found ", len(speakers), "speakers")
    print(speakers)
    wav_list = []
    train_data = []
    val_data = []
    for id_ in tqdm(speakers):
        utterences = ['/'.join(x.parts[-2:]) for x in id_.glob("*.pt")]
        print(utterences)
        random.shuffle(utterences)
        split_len = len(utterences) - int(len(utterences) * val_ratio)
        split_train = utterences[:split_len + 1]
        split_val = utterences[split_len+1:]
        train_data.extend(split_train)
        val_data.extend(split_val)
        # split_list.extend((np.random.rand(len(utterences)) > val_ratio).astype(int))
        wav_list.extend(utterences)
    # save split.txt
    with open(train_path, "w") as split:
        for i, n in enumerate(train_data):
            split.write("%d %s\n" % (n, train_data[i]))
    
    # with open(val_path, "w") as split:
    #     for i, n in enumerate(val_data):
    #         split.write("%d %s\n" % (n, val_data[i]))

    # print(len(split_list)," files split into train/val")

In [236]:
from pathlib import Path

train_dir = Path("/Users/ujinne/python/Hackathon/speaker_recognition/data/sample/feature/train/")
train_path = Path("/Users/ujinne/python/Hackathon/speaker_recognition/data/sample/feature/train/train_data.txt")
val_path = Path("/Users/ujinne/python/Hackathon/speaker_recognition/data/sample/feature/train/val_data.txt")



split_train_val(train_dir, train_path, val_path)

Found  3 speakers
[PosixPath('/Users/ujinne/python/Hackathon/speaker_recognition/data/sample/feature/train/0CAE3COBqO'), PosixPath('/Users/ujinne/python/Hackathon/speaker_recognition/data/sample/feature/train/7K4iJllpGl'), PosixPath('/Users/ujinne/python/Hackathon/speaker_recognition/data/sample/feature/train/6cmw8oKHST')]


100%|██████████| 3/3 [00:00<00:00, 1818.08it/s]

['0CAE3COBqO/0CAE3COBqO_0.pt', '0CAE3COBqO/0CAE3COBqO_1.pt']
['7K4iJllpGl/7K4iJllpGl_1.pt', '7K4iJllpGl/7K4iJllpGl_0.pt']
['6cmw8oKHST/6cmw8oKHST_0.pt', '6cmw8oKHST/6cmw8oKHST_1.pt']


TypeError: %d format: a number is required, not str

In [231]:
np.random.rand(6) > 0.2

array([ True,  True, False,  True,  True,  True])

In [232]:
int(1.2)

1

In [257]:
class GE2ELoss(nn.Module):

    def __init__(self, init_w=10.0, init_b=-5.0, loss_method='softmax'):
        '''
        Implementation of the Generalized End-to-End loss defined in https://arxiv.org/abs/1710.10467 [1]
        Accepts an input of size (N, M, D)
            where N is the number of speakers in the batch,
            M is the number of utterances per speaker,
            and D is the dimensionality of the embedding vector (e.g. d-vector)
        Args:
            - init_w (float): defines the initial value of w in Equation (5) of [1]
            - init_b (float): definies the initial value of b in Equation (5) of [1]
        '''
        super(GE2ELoss, self).__init__()
        self.w = nn.Parameter(torch.tensor(init_w))
        self.b = nn.Parameter(torch.tensor(init_b))
        self.loss_method = loss_method

        assert self.loss_method in ['softmax', 'contrast']

        if self.loss_method == 'softmax':
            self.embed_loss = self.embed_loss_softmax
        if self.loss_method == 'contrast':
            self.embed_loss = self.embed_loss_contrast

    def calc_new_centroids(self, dvecs, centroids, spkr, utt):
        '''
        Calculates the new centroids excluding the reference utterance
        '''
        excl = torch.cat((dvecs[spkr,:utt], dvecs[spkr,utt+1:]))
        excl = torch.mean(excl, 0)
        new_centroids = []
        for i, centroid in enumerate(centroids):
            if i == spkr:
                new_centroids.append(excl)
            else:
                new_centroids.append(centroid)
        return torch.stack(new_centroids)

    def calc_cosine_sim(self, dvecs, centroids):
        '''
        Make the cosine similarity matrix with dims (N,M,N)
        '''
        cos_sim_matrix = []
        for spkr_idx, speaker in enumerate(dvecs):
            cs_row = []
            for utt_idx, utterance in enumerate(speaker):
                new_centroids = self.calc_new_centroids(dvecs, centroids, spkr_idx, utt_idx)
                # vector based cosine similarity for speed
                cs_row.append(torch.clamp(torch.mm(utterance.unsqueeze(1).transpose(0,1), new_centroids.transpose(0,1)) / (torch.norm(utterance) * torch.norm(new_centroids, dim=1)), 1e-6))
            cs_row = torch.cat(cs_row, dim=0)
            cos_sim_matrix.append(cs_row)
        return torch.stack(cos_sim_matrix)

    def embed_loss_softmax(self, dvecs, cos_sim_matrix):
        '''
        Calculates the loss on each embedding $L(e_{ji})$ by taking softmax
        '''
        N, M, _ = dvecs.shape
        L = []
        for j in range(N):
            L_row = []
            for i in range(M):
                L_row.append(-F.log_softmax(cos_sim_matrix[j,i], 0)[j])
            L_row = torch.stack(L_row)
            L.append(L_row)
        return torch.stack(L)

    def embed_loss_contrast(self, dvecs, cos_sim_matrix):
        ''' 
        Calculates the loss on each embedding $L(e_{ji})$ by contrast loss with closest centroid
        '''
        N, M, _ = dvecs.shape
        L = []
        for j in range(N):
            L_row = []
            for i in range(M):
                centroids_sigmoids = torch.sigmoid(cos_sim_matrix[j,i])
                excl_centroids_sigmoids = torch.cat((centroids_sigmoids[:j], centroids_sigmoids[j+1:]))
                L_row.append(1. - torch.sigmoid(cos_sim_matrix[j,i,j]) + torch.max(excl_centroids_sigmoids))
            L_row = torch.stack(L_row)
            L.append(L_row)
        return torch.stack(L)

    def forward(self, dvecs):
        '''
        Calculates the GE2E loss for an input of dimensions (num_speakers, num_utts_per_speaker, dvec_feats)
        '''
        #Calculate centroids
        centroids = torch.mean(dvecs, 1)

        #Calculate the cosine similarity matrix
        cos_sim_matrix = self.calc_cosine_sim(dvecs, centroids)
        torch.clamp(self.w, 1e-6)
        cos_sim_matrix = cos_sim_matrix * self.w + self.b
        L = self.embed_loss(dvecs, cos_sim_matrix)
        return L.sum()


In [258]:
import torch

criterion = GE2ELoss(init_w=10.0, init_b=-5.0, loss_method='softmax') #for softmax loss
# criterion = GE2ELoss(init_w=10.0, init_b=-5.0, loss_method='contrast') #for contrast loss

N = 64 #Number of speakers in a batch
M = 10 #Number of utterances for each speaker
D = 256 #Dimensions of the speaker embeddings, such as a d-vector or x-vector

test_input = torch.rand(N, M, D)
loss = criterion(test_input) #output is a scalar
# loss.backward()
print(loss)

tensor(2670.7954, grad_fn=<SumBackward0>)


In [394]:
tensor = torch.rand(125, 40)
sampling_frame_length = 157

In [387]:
len_time = tensor.size(0)
feature = tensor.size(1)
# print(len_time)
# if len_time < sampling_frame_length:
#     # print(f'prev_tensor:{utt_tensor.size(0)}')
#     # tensor = F.interpolate(tensor.unsqueeze(0), (1, sampling_frame_length,10))
#     tensor = F.pad(tensor, (0, 0, , 0), "constant", 0)
#     print(tensor.shape)
#     len_time = tensor.size(0)
#     print(f'changed_tensor:{len_time}')
# len_time = len_time - sampling_frame_length
# # print(f'len_time:{len_time}')
# start_time = random.randint(0, len_time)
# end_time = start_time + sampling_frame_length

In [395]:
padder = torch.zeros(sampling_frame_length-len_time, feature)
padder_a = torch.cat([tensor, padder], dim=0)

In [397]:
padder_a.shape

torch.Size([157, 40])

In [404]:
path = './text/'
os.makedirs(path, exist_ok=True)

# Predict

In [405]:
import pandas as pd

In [423]:
train_meta = pd.read_csv('/Users/ujinne/python/Hackathon/speaker_recognition/data/sample/feature/train_meta.csv')
spk_ids = train_meta['id'].tolist()

In [428]:
spk_ids[:-1]

['0CAE3COBqO', '6cmw8oKHST', '7K4iJllpGl']

In [429]:
classes = {}

In [430]:
for spk_id in spk_ids[:-1]:
    classes[spk_id] = 3920

In [432]:
classes

{'0CAE3COBqO': 3920, '6cmw8oKHST': 3920, '7K4iJllpGl': 3920}

In [2]:
import torch
a = [torch.rand(1, 325, 40).squeeze(0) for _ in range(64)]


/Users/ujinne/opt/anaconda3/envs/m2m/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
a = torch.stack(a)
a.squeeze(0).shape

torch.Size([64, 325, 40])

In [463]:
b = torch.rand(1, 325, 40).squeeze(0)

In [474]:
c = b.unsqueeze(0)

torch.Size([1, 40])

In [476]:
b.shape

torch.Size([325, 40])

In [9]:
a = torch.rand(64, 390, 256)

In [14]:
a.transpose(1,2)[0].squeeze(0).detach().numpy().shape

(256, 390)

In [17]:
tensor1 = torch.rand((4,2,3,100))
tensor2 = torch.rand((4,2,3,100))
tensor1 = tensor1.transpose(1,3)
tensor2 = tensor2.transpose(1,3)
dist = torch.nn.functional.pairwise_distance(tensor1, tensor2)
dist = dist.transpose(1,2)

In [19]:
dist.shape

torch.Size([4, 2, 3])

In [32]:
a = torch.rand(3, 256)
a = torch.mean(a, 0)
a = a.unsqueeze(0)
a.shape

torch.Size([1, 256])

In [9]:
import torch
b = torch.rand(1, 256)

In [10]:
b = torch.cat([b, b], dim=0)
b = torch.cat([b, b], dim=0)
b = torch.cat([b, b], dim=0)

b.shape

torch.Size([8, 256])

In [20]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300, random_state=256)
transformed = tsne.fit_transform(cent.numpy())

[t-SNE] Computing 2 nearest neighbors...
[t-SNE] Indexed 3 samples in 0.000s...
[t-SNE] Computed neighbors for 3 samples in 0.004s...
[t-SNE] Computed conditional probabilities for sample 3 / 3
[t-SNE] Mean sigma: 1125899906842624.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 35.589149
[t-SNE] KL divergence after 300 iterations: 0.000222


In [21]:
transformed.shape

(3, 2)

In [12]:
print(transformed[:, 0], transformed[:, 1])

[ 18.994495 -19.368113   9.341522  33.315666 -10.998646  36.355682
  12.294693  -5.726732] [ 16.530998   -7.064364  -10.278559  -25.611708  -30.41482    -0.9661348
 -38.797516   13.746578 ]


In [75]:
test = []
for i in range(5): 
    name = ["dksjg", "skuhw", "eouw", "wogs", "skdjg"]
    add =  {"spk" : name[i], "cent" : torch.rand(1, 256), "max_di" : torch.tensor(322.302)}
    test.append(add)

In [76]:
test[0]


[{'spk': 'dksjg',
  'cent': array([[0.32747275, 0.95415705, 0.05116272, 0.28453445, 0.61130506,
          0.8060788 , 0.2526006 , 0.6435509 , 0.906244  , 0.38056356,
          0.9541275 , 0.90351903, 0.60839957, 0.42306834, 0.17191082,
          0.61234015, 0.26895785, 0.34389615, 0.13387328, 0.79870725,
          0.8946097 , 0.27146572, 0.6016372 , 0.2106511 , 0.2703529 ,
          0.54561645, 0.44326657, 0.3352247 , 0.8531119 , 0.84005004,
          0.94773555, 0.77805305, 0.6083294 , 0.42409253, 0.43590403,
          0.90867704, 0.40833598, 0.39948463, 0.5250719 , 0.28788692,
          0.8056392 , 0.5286517 , 0.9910353 , 0.5611106 , 0.5842323 ,
          0.64239454, 0.90982074, 0.11169189, 0.72790104, 0.21077943,
          0.3913222 , 0.11307943, 0.22355461, 0.67547894, 0.9325117 ,
          0.7340604 , 0.51652324, 0.26159132, 0.44790006, 0.05375791,
          0.85780513, 0.03015476, 0.5606157 , 0.32705408, 0.23939067,
          0.20225978, 0.24467689, 0.73079526, 0.48709995, 0.6680

In [18]:
dvecs = torch.rand(3, 140, 256)

In [19]:
cent = torch.mean(dvecs, dim=1)
cent.shape

torch.Size([3, 256])

In [17]:
cent

tensor([0.4970, 0.4703, 0.5007])

In [25]:
import glob
test_dir = './data/sample/feature/test'

utts = [f for f in glob.glob(test_dir + '/*.pt') if os.path.isfile(f)]

In [27]:
recording = [f.split('/')[-1].replace(".pt", ".wav") for f in utts]

In [28]:
recording

['owemm8FAOM_0.wav',
 'owemm8FAOM_1.wav',
 'mtNgnWp7R8_0.wav',
 'mtNgnWp7R8_1.wav']

In [54]:
import torch
# from torchmetrics.functional import pairwise_euclidean_distance

x = torch.rand(1, 256)
y = torch.rand(1, 256)

# print(pairwise_euclidean_distance(x, y))
print(torch.cdist(x, y, p=2).shape)

torch.Size([1, 1])


In [42]:
print(torch.sqrt(((x-y)**2).sum(axis=1)))

tensor([6.3092])


In [41]:
dist = torch.nn.PairwiseDistance(p=2)
print(dist(x, y))

tensor([6.3092])


In [44]:
import numpy as np
max_dis = np.array(322.302)

In [55]:
dist = torch.cdist(x, y, p=2)

In [58]:
dist

tensor([[6.9463]])

In [53]:
a = dist.squeeze(0)

In [59]:
a

tensor([6.3092])

In [57]:
lst = [dist, a]
max(lst)

tensor([[6.9463]])

# VAD

In [2]:
import torchaudio
import webrtcvad
vad = webrtcvad.Vad(2)

/Users/ujinne/opt/anaconda3/envs/m2m/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
import librosa
import torch
import torchaudio
import numpy as np
from typing import Optional, Union
from pathlib import Path

sampling_rate = 16000

def preprocess_wav(fpath_or_wav: Union[str, Path, np.ndarray],
                   source_sr: Optional[int] = None):
    # Load the wav from disk if needed
    if isinstance(fpath_or_wav, str) or isinstance(fpath_or_wav, Path):
        wav, source_sr = librosa.load(fpath_or_wav, sr=None)
    else:
        wav = fpath_or_wav

    # Resample the wav if needed
    if source_sr is not None and source_sr != sampling_rate:
        wav = librosa.resample(wav, source_sr, sampling_rate)
    # print(type(wav))

    # trim audio
    wav = torchaudio.functional.vad(torch.from_numpy(wav), source_sr)

    # # Apply the preprocessing: normalize volume and shorten long silences
    # wav = normalize_volume(wav, audio_norm_target_dBFS, increase_only=True)

    return np.array(wav)

In [41]:
path = '/Users/ujinne/python/Hackathon/speaker_recognition/data/sample/train/mtNgnWp7R8/mtNgnWp7R8_1.wav'
wav = preprocess_wav(path)
wav.shape


<class 'numpy.ndarray'>


(50415,)

In [42]:
from scipy.io.wavfile import write

write("example.wav", 16000, wav)

In [45]:
wav, sr = torchaudio.load(path)
wav.shape

torch.Size([1, 69615])

In [46]:
wav, source_sr = librosa.load(path, sr=None)
wav.shape

(69615,)

In [50]:
dic = {'a': 1, 'b': 2, 'c':3}
dic['e'] = len(dic)+1
dic

{'a': 1, 'b': 2, 'c': 3, 'e': 4}

In [65]:
dic.get('e', 'a')

4

In [124]:
out = torch.rand(1, 3124)
# tgt = torch.randint(0, 10, (640,))
tgt = torch.randint(0, 3124, (640,))
criterion = torch.nn.CrossEntropyLoss()

In [85]:

input = torch.randn(640, 3183)
labels = torch.randint(0, 3183, (640,))
target = torch.zeros_like(input)
target.scatter_(1, labels.unsqueeze(1), 1)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [136]:
probs = torch.softmax(out)
# probs = probs.max(dim=1)
# probs = probs.argmax(dim=1)
probs
# torch.max(out.softmax(-1)).item()

TypeError: softmax() received an invalid combination of arguments - got (Tensor), but expected one of:
 * (Tensor input, int dim, torch.dtype dtype)
 * (Tensor input, name dim, *, torch.dtype dtype)


In [133]:
 if torch.max(values.softmax(-1)).item() < threshold:
    prediction = 'unknown'

NameError: name 'values' is not defined

In [102]:
def  accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        print(f'pred.shape:{pred.shape}')
        pred = pred.t() #transpose
        print(f'pred.shape:{pred.shape}')
        print(f'target.shape:{target.shape}')
        correct = pred.eq(target.view(1, -1).expand_as(pred))
        print(f'target: {target.view(1, -1).expand_as(pred)}')
        print(f'pred: {pred}')
        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))

        return res

In [ ]:
def predict(cfg, output, topk=(1,), threshold=0.5):
    dataset_root = Path(cfg.DATASET.DATA_DIR)
    with torch.no_grad():
        maxk = max(topk)
        batch_size = 1

        values, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()

        # Getting the most probable speaker_id(str)
        prediction = find_id(dataset_root, pred[0][0].item())

        # Custom
        if torch.max(values.softmax(-1)).item() < threshold:
            prediction = 'unknown'

        return prediction

In [142]:
topk = (1,)
maxk = max(topk)
values, pred = out.topk(maxk, 1, True, True)
print(f'values:{values}')
print(f'pred:{pred}')
print(f'pred:{pred.shape}')
pred = pred.t()

torch.max(values.softmax(-1)).item()

values:tensor([[0.9995]])
pred:tensor([[481]])
pred:torch.Size([1, 1])


1.0

In [141]:
torch.max(values.softmax(-1)).item()

1.0

In [144]:
os.listdir('/Users/ujinne/python/Hackathon/Lipreading/')

['nsml_package.txt',
 'baseline_code',
 '.DS_Store',
 'main_stt1.ipynb',
 'requirements.txt',
 'Lipreading_final',
 'Makefile',
 'main_stt1.py',
 'AudioVisualLoss.ipynb',
 'stt_model.py',
 '__pycache__',
 'setup.py',
 'conformer',
 'main_stt2.py',
 'fps_changer.py',
 'checkpoint.tar',
 '.ipynb_checkpoints',
 'data',
 'data.py']